# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [3]:
#- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
a = [1.0,1.4142135623730951,1.7320508075688772,2.0,2.23606797749979]

print(sum(a))

8.382332347441762


- Answer Question 1: **8.382**

In [4]:
#Question2:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


- Answer question2: **3.605**

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [15]:
import dlt
import duckdb

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')



# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline.run(people_1(),
					table_name="person_append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.68 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Rodolfo\DEZoomcamp\workshop1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707767765.2390506 is LOADED and contains no failed jobs


In [16]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person              │
│ person_append       │
└─────────────────────┘

In [17]:
print("\n\n\n Person table below: After Loading first generator")
person_append = conn.sql("SELECT * FROM person_append").df()
display(person_append)




 Person table below: After Loading first generator


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707767765.2390506,BmRIzDjxTviwWg
1,2,Person_2,27,City_A,1707767765.2390506,B9d7QkPL0Navcw
2,3,Person_3,28,City_A,1707767765.2390506,JwE7jqDtUkhgNA
3,4,Person_4,29,City_A,1707767765.2390506,OypJs7puW0Ff3w
4,5,Person_5,30,City_A,1707767765.2390506,zKlP8oNWcMwFXQ


In [18]:
print("\n\n\n Sum of Ages")
display(conn.sql("SELECT SUM(age) FROM person_append").df())




 Sum of Ages


,sum(age)
0,140.0


- **Appending Second generator**

In [19]:
info = pipeline.run(people_2(),
					table_name="person_append",
					write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.47 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Rodolfo\DEZoomcamp\workshop1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707767799.9368403 is LOADED and contains no failed jobs


In [20]:
print("\n\n\n Person table below: After appending secong generator")
person_append = conn.sql("SELECT * FROM person_append").df()
display(person_append)




 Person table below: After appending secong generator


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707767765.2390506,BmRIzDjxTviwWg,None
1,2,Person_2,27,City_A,1707767765.2390506,B9d7QkPL0Navcw,None
2,3,Person_3,28,City_A,1707767765.2390506,JwE7jqDtUkhgNA,None
3,4,Person_4,29,City_A,1707767765.2390506,OypJs7puW0Ff3w,None
4,5,Person_5,30,City_A,1707767765.2390506,zKlP8oNWcMwFXQ,None
5,3,Person_3,33,City_B,1707767799.9368403,eUYlKmSU+NkA2g,Job_3
6,4,Person_4,34,City_B,1707767799.9368403,0w7G6L/4yK4LBA,Job_4
7,5,Person_5,35,City_B,1707767799.9368403,yDDH8r1eOQwopg,Job_5
8,6,Person_6,36,City_B,1707767799.9368403,+7at2ykvDoASwQ,Job_6
9,7,Person_7,37,City_B,1707767799.9368403,SrC8YkXrDGhZ8g,Job_7


**After correctly appending the data, calculate the sum of all ages of people.**

In [21]:
print("\n\n\n Sum of Ages")
display(conn.sql("SELECT SUM(age) FROM person_append").df())




 Sum of Ages


,sum(age)
0,353.0


- **Answer question 3:** 353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [6]:
import dlt
import duckdb

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')



# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline.run(people_1(),
					table_name="person",
					write_disposition="merge",
                    primary_key="ID")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 3.12 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Rodolfo\DEZoomcamp\workshop1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707765817.2640936 is LOADED and contains no failed jobs


In [7]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ person              │
└─────────────────────┘

In [8]:
print("\n\n\n Person table below: After Loading first generator")
person = conn.sql("SELECT * FROM person").df()
display(person)




 Person table below: After Loading first generator


,id,name,age,city,_dlt_load_id,_dlt_id
0,5,Person_5,30,City_A,1707765817.2640936,WOcRD9DtqlCYlg
1,2,Person_2,27,City_A,1707765817.2640936,pBJbrypsCmzang
2,4,Person_4,29,City_A,1707765817.2640936,qvoOwy5MBHqH1w
3,1,Person_1,26,City_A,1707765817.2640936,1+SncaUzOXCN1g
4,3,Person_3,28,City_A,1707765817.2640936,CA23Qf54bg8F1g


In [9]:
print("\n\n\n Sum of Ages")
display(conn.sql("SELECT SUM(age) FROM person").df())





 Sum of Ages


,sum(age)
0,140.0


#### Merging the second generator

In [10]:
info = pipeline.run(people_2(),
					table_name="person",
					write_disposition="merge",
                    primary_key="ID")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 1.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Rodolfo\DEZoomcamp\workshop1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707766625.488739 is LOADED and contains no failed jobs


In [14]:
print("\n\n\n Person table below: After merging secong generator")
person = conn.sql("SELECT * FROM person").df()
display(person)




 Person table below: After merging secong generator


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707765817.2640936,pBJbrypsCmzang,None
1,1,Person_1,26,City_A,1707765817.2640936,1+SncaUzOXCN1g,None
2,7,Person_7,37,City_B,1707766625.488739,ESilN1ft7HluEw,Job_7
3,4,Person_4,34,City_B,1707766625.488739,Z0ah92gfQr67YQ,Job_4
4,5,Person_5,35,City_B,1707766625.488739,iFqFl84gveUVdA,Job_5
5,8,Person_8,38,City_B,1707766625.488739,qAAu+mmxKTlJ2w,Job_8
6,3,Person_3,33,City_B,1707766625.488739,QJNDvCSV+HTJPQ,Job_3
7,6,Person_6,36,City_B,1707766625.488739,9bpOL/ScPB1ZgA,Job_6


**After correctly merging the data, calculate the sum of all ages of people.**

In [13]:
print("\n\n\n Sum of Ages")
display(conn.sql("SELECT SUM(age) FROM person").df())




 Sum of Ages


,sum(age)
0,266.0


- **Answer question 4:** 266

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX